<a href="https://colab.research.google.com/github/akuma527/IProjects/blob/master/I%26D_Whale/Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install -U tqdm

     |████████████████████████████████| 61kB 2.0MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/I&D_Whale/data/

/content/drive/My Drive/Colab Notebooks/I&D_Whale/data


## Loading a Pretrained VGG16

In [0]:
import torch
import numpy as np
import torchvision.models as models
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image

In [5]:
model = models.vgg16(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
# model = models.resnet34(pretrained=True, progress=True)

Image Preprocessing Function

In [0]:
import pandas as pd
import os
import cv2

Removing Classification Layer

In [0]:
model.classifier = torch.nn.Sequential(*[model.classifier[i] for i in range(4)])

# Convert Images to Embeddings

In [0]:
imsize = 256
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    v = model(image).detach().numpy().flatten().reshape(1,-1)
    return v #assumes that you're using GPU

In [0]:
# a = t1[['Image', 'fil_path']]
# a['type'] = 'train'

In [0]:
# b = t2[['Image', 'fil_path']]
# b['type'] = 'test'

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['type'] = 'train'
test['type'] = 'test'
df = pd.concat([train, test], ignore_index=True, sort=False)
# df = pd.concat([a, b], ignore_index=True, sort=False)

In [13]:
df.head()

,Image,Id,fil_path,type
0,PM-WWA-20050416-037.jpg,-1,train/-1/PM-WWA-20050416-037.jpg,train
1,PM-WWA-20050513-229.jpg,-1,train/-1/PM-WWA-20050513-229.jpg,train
2,PM-WWA-20050513-243.jpg,-1,train/-1/PM-WWA-20050513-243.jpg,train
3,PM-WWA-20050520-134.jpg,-1,train/-1/PM-WWA-20050520-134.jpg,train
4,PM-WWA-20050612-047.jpg,-1,train/-1/PM-WWA-20050612-047.jpg,train


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
from tqdm.auto import tqdm
tqdm.pandas()

In [0]:
df['vector'] = df['fil_path'].progress_apply(image_loader)

In [0]:
import pickle
with open('Fruits_ORG.obj', 'wb') as fp:
    pickle.dump(df, fp)

In [0]:
k_df = pd.DataFrame(k.items(), columns=['fil_path', 'embeddings'])

In [0]:
k_df.set_index('fil_path', inplace=True)

In [0]:
p = []
for i in tqdm_notebook(range(len(k_df))):
    for j in range(len(k_df)):
        p.append(cosine_similarity(k_df['embeddings'].iloc[i], k_df['embeddings'].iloc[j]))

In [0]:
len(p)